In [1]:
!pip install vadersentiment
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 21.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 9.8 MB/s 
     |████████████████████████████████| 662 kB 59.0 MB/s 
     |████████████████████████████████| 5.2 MB 63.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 85 kB 5.0 MB/s 
     |████████████████████████████████| 88 kB 8.2 MB/s 
     |████████████████████████████████| 5.8 MB 60.5 MB/s 
     |████████████████████████████████| 1.3 MB 61.1 MB/s 
     |████████████████████████████████| 182 kB 79.5 MB/s 
     |████████████████████████████████| 7.6 MB 49.0 MB/s 
     |████████████████████████████████| 1.1 MB 66.3 MB/s 
  Created wheel for hdbscan: filename=hdbsca

In [2]:
import pandas as pd
import numpy as np
import gensim
from nltk.stem import WordNetLemmatizer, SnowballStemmer
np.random.seed(10)
import nltk
from bertopic import BERTopic
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
import seaborn as sns
import plotly.express as px
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [3]:
def clean_text(text):
    text = text.lower() # lowercase everything
    text = text.encode('ascii', 'ignore').decode()  # remove unicode characters
    text = re.sub(r'https*\S+', ' ', text) # remove links
    text = re.sub(r'http*\S+', ' ', text)
    # cleaning up text
    text = re.sub(r'\'\w+', '', text) 
    text = re.sub(r'\w*\d+\w*', '', text)
    text = re.sub(r'\s{2,}', ' ', text)
    text = re.sub(r'\s[^\w\s]\s', '', text)
    text = re.sub("[\(\[].*?[\)\]]", "", text)
    text = re.sub('[^a-zA-Z0-9 \.]', '', text)
    return text

In [4]:
def lemmatize(text):
  return WordNetLemmatizer().lemmatize(text, pos='v')
def preprocess(text):
    if type(text)!=str:
      return ''
    result = []
    text = text.replace('[removed]', '').replace('[deleted]', '')
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2:
            result.append(lemmatize(token))
    return ' '.join(result)

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
import pandas as pd
file_name = "/content/drive/MyDrive/raw_reddit_news_posts_comments.csv"
df = pd.read_csv(file_name)
df.head()

,Unnamed: 0,id,subreddit,author,created_utc,domain,url,title,num_comments,selftext,score,comments,url_content
0,0,eib002,worldnews,DoremusJessup,2020-01-01 00:00:08,reuters.com,https://www.reuters.com/article/us-india-citiz...,Thousands of Indians ushered in the New Year b...,17,NaN,1,[deleted]...Because most Indians are not Musli...,NEW DELHI (Reuters) - Thousands of Indians ush...
1,1,eib9ut,worldnews,skippy_smooth,2020-01-01 00:21:23,bgr.com,https://bgr.com/2019/12/31/china-pneumonia-sic...,A scary unidentified virus is spreading in China,70,NaN,1,"[removed]...Ah, a fellow time traveler....Well...","Dozens of residents of Wuhan, the capital city..."
2,2,eibebq,worldnews,vannybros,2020-01-01 00:31:16,cnn.com,https://www.cnn.com/2019/12/31/politics/north-...,Kim Jong Un warns hostile US policy means ther...,10,NaN,1,This guy...Trump will comfort Kim....Being tha...,Washington CNN —\n\nNorth Korean Leader Kim Jo...
3,3,eibo17,worldnews,OldFashionedJizz,2020-01-01 00:53:25,theguardian.com,https://www.theguardian.com/world/2019/dec/31/...,"North Korean leader to end missile test ban, c...",7,NaN,1,[removed]...That's a choice of course. We can ...,Kim Jong-un has signalled that North Korea wil...
4,4,eibo8f,worldnews,didnt_see_that_tree,2020-01-01 00:53:55,independent.co.uk,https://www.independent.co.uk/life-style/gadge...,Snapchat has stopped working for users around ...,8,NaN,1,Oh The Humanity ! ! !...OMG....what will teena...,For free real time breaking news alerts sent s...


In [26]:
print(df.shape)
df.fillna("", inplace=True)
len(df[df.id==""])

(28800, 13)


0

In [27]:
# df[df['selftext'].isna()] = ""
df["title"] = df["title"].astype(str) 
df["selftext"] = df["selftext"].astype(str)
df["title"] = df["title"] + " " + df["selftext"]

In [28]:
df['cleaned_title'] = df.title.apply(preprocess)
titles = df["cleaned_title"].astype(str).to_list()

In [73]:
topic_model = BERTopic(min_topic_size=100, n_gram_range=(1,3), nr_topics='auto', verbose=True)
topics, _ = topic_model.fit_transform(titles)

Batches:   0%|          | 0/900 [00:00<?, ?it/s]

2022-12-02 17:43:45,307 - BERTopic - Transformed documents to Embeddings
2022-12-02 17:44:17,374 - BERTopic - Reduced dimensionality
2022-12-02 17:44:21,764 - BERTopic - Clustered reduced embeddings
2022-12-02 17:44:28,308 - BERTopic - Reduced number of topics from 48 to 44


In [76]:
topic_model.save("/content/drive/MyDrive/topic_news_model")

In [75]:
barchart = topic_model.visualize_barchart(top_n_topics=10, n_words=10, height=400)
barchart

In [77]:
barchart.write_html("/content/drive/MyDrive/news_word_freq.html")

In [74]:
intertopic_dist = topic_model.visualize_topics()
intertopic_dist

In [78]:
intertopic_dist.write_html("/content/drive/MyDrive/news_intertopic_distance.html")

In [79]:
freq = topic_model.get_topic_info()
freq.head(10)

,Topic,Count,Name
0,-1,9774,-1_coronavirus_covid_say_test
1,0,3192,0_china_coronavirus_chinese_wuhan
2,1,1720,1_pakistan_india_school_coronavirus
3,2,1711,2_iran_iraq_soleimani_iranian
4,3,823,3_putin_death_toll_death toll
5,4,744,4_trump_coronavirus_fund_white house
6,5,649,5_mask_face mask_face_ventilators
7,6,638,6_coronavirus_case_coronavirus case_pandemic
8,7,552,7_israel_palestinian_netanyahu_israeli
9,8,544,8_dog_animals_meat_cat


In [80]:
topic_nr = freq.iloc[3]["Topic"]  # We select a frequent topic
topic_model.get_topic(topic_nr)

[('iran', 0.0459590766647272),
 ('iraq', 0.026668650975522564),
 ('soleimani', 0.025269840221098537),
 ('iranian', 0.020610758436719376),
 ('kill', 0.020458673485115948),
 ('trump', 0.018488021515966777),
 ('troop', 0.01653631019162259),
 ('attack', 0.015561837258337229),
 ('strike', 0.014052325059478708),
 ('iraqi', 0.01380949543753012)]

In [81]:
freq["word_strength"] = freq.Topic.apply(topic_model.get_topic)
freq.head()

,Topic,Count,Name,word_strength
0,-1,9774,-1_coronavirus_covid_say_test,"[(coronavirus, 0.012105833927353142), (covid, ..."
1,0,3192,0_china_coronavirus_chinese_wuhan,"[(china, 0.04591752136849972), (coronavirus, 0..."
2,1,1720,1_pakistan_india_school_coronavirus,"[(pakistan, 0.019609116424236742), (india, 0.0..."
3,2,1711,2_iran_iraq_soleimani_iranian,"[(iran, 0.0459590766647272), (iraq, 0.02666865..."
4,3,823,3_putin_death_toll_death toll,"[(putin, 0.02687867568045271), (death, 0.02319..."


In [82]:
freq.to_csv("/content/drive/MyDrive/news_topic_frequency.csv", index=False)

In [83]:
df['created_utc'] = pd.to_datetime(df['created_utc'])
df["created_dt_month"] = df['created_utc'].dt.strftime('%m-%Y')
titles = df["cleaned_title"].astype(str).to_list()
dates = df['created_utc']
topics_over_time = topic_model.topics_over_time(docs=titles,  
                                                # topics=topics, 
                                                timestamps=dates, 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=30)

30it [01:10,  2.34s/it]


In [84]:
tot = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)
tot

In [85]:
tot.write_html("/content/drive/MyDrive/news_topics_over_time.html")

In [86]:
topics_over_time.to_csv("/content/drive/MyDrive/topics_over_time_news.csv", index=False)
representative_docs = topic_model.get_representative_docs()

docs_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in representative_docs.items() ])).T
docs_df.index.name = "topic_nr"
docs_df.to_csv("/content/drive/MyDrive/representative_docs_news.csv")
docs_df.head()

,0,1,2,3,4,5,6,7,8
topic_nr,,,,,,,,,
1,god muslims pakistan flout coronavirus ban mos...,modi want people religious freedom say preside...,report claim pakistani doctor affect covid hea...,german finance minister commit suicide virus c...,school shut friday exams postpone,ireland set close school universities coronavirus,NaN,NaN,NaN
0,china tell confident slay devil virus,chinese man bust paddle wooden tub yangtze riv...,hong kong hold major protest month hong kong p...,NaN,NaN,NaN,NaN,NaN,NaN
14,kim jong illness rumour deny south korea,taiwan intelligence chief say north korea kim ...,south korea say north korea kim seriously ill ...,NaN,NaN,NaN,NaN,NaN,NaN
26,youtube ceo ban coronavirus content guidelines,implement article copyright law,plandemic free youtube censorship,NaN,NaN,NaN,NaN,NaN,NaN
2,lead military coalition say withdraw iraq coun...,trump address nation amid showdown iran,iran send multiple message attack,turkish syrian soldier kill battle control idl...,turkey repeat work group offer solve row russi...,provide air support turkey syria pentagon,NaN,NaN,NaN


In [87]:
df['topic'] = df['cleaned_title'].apply(lambda x: topic_model.find_topics(x)[0][0])
df.head()

,Unnamed: 0,id,subreddit,author,created_utc,domain,url,title,num_comments,selftext,score,comments,url_content,cleaned_title,created_dt_month,topic,cleaned_comments,comment_sentiment
0,0,eib002,worldnews,DoremusJessup,2020-01-01 00:00:08,reuters.com,https://www.reuters.com/article/us-india-citiz...,Thousands of Indians ushered in the New Year b...,17,,1,[deleted]...Because most Indians are not Musli...,NEW DELHI (Reuters) - Thousands of Indians ush...,thousands indians usher new year demonstrate c...,01-2020,32,indians muslims million muslim feel real citiz...,0.9875
1,1,eib9ut,worldnews,skippy_smooth,2020-01-01 00:21:23,bgr.com,https://bgr.com/2019/12/31/china-pneumonia-sic...,A scary unidentified virus is spreading in China,70,,1,"[removed]...Ah, a fellow time traveler....Well...","Dozens of residents of Wuhan, the capital city...",scary unidentified virus spread china,01-2020,0,fellow time traveler good turn interest gamepl...,0.9705
2,2,eibebq,worldnews,vannybros,2020-01-01 00:31:16,cnn.com,https://www.cnn.com/2019/12/31/politics/north-...,Kim Jong Un warns hostile US policy means ther...,10,,1,This guy...Trump will comfort Kim....Being tha...,Washington CNN —\n\nNorth Korean Leader Kim Jo...,kim jong warn hostile policy mean korean penin...,01-2020,14,guy trump comfort kim intentions nuke meaningl...,-0.7849
3,3,eibo17,worldnews,OldFashionedJizz,2020-01-01 00:53:25,theguardian.com,https://www.theguardian.com/world/2019/dec/31/...,"North Korean leader to end missile test ban, c...",7,,1,[removed]...That's a choice of course. We can ...,Kim Jong-un has signalled that North Korea wil...,north korean leader end missile test ban claim...,01-2020,14,choice course agree big problem north korea pu...,-0.9791
4,4,eibo8f,worldnews,didnt_see_that_tree,2020-01-01 00:53:55,independent.co.uk,https://www.independent.co.uk/life-style/gadge...,Snapchat has stopped working for users around ...,8,,1,Oh The Humanity ! ! !...OMG....what will teena...,For free real time breaking news alerts sent s...,snapchat stop work users world tens thousands ...,01-2020,26,humanity omg teenagers world snap corner carpe...,-0.8176


In [88]:
df['cleaned_comments'] = df['comments'].map(preprocess)
df.to_csv("/content/drive/MyDrive/reddit_news_data_with_topics.csv", index=False)
df.head()

,Unnamed: 0,id,subreddit,author,created_utc,domain,url,title,num_comments,selftext,score,comments,url_content,cleaned_title,created_dt_month,topic,cleaned_comments,comment_sentiment
0,0,eib002,worldnews,DoremusJessup,2020-01-01 00:00:08,reuters.com,https://www.reuters.com/article/us-india-citiz...,Thousands of Indians ushered in the New Year b...,17,,1,[deleted]...Because most Indians are not Musli...,NEW DELHI (Reuters) - Thousands of Indians ush...,thousands indians usher new year demonstrate c...,01-2020,32,indians muslims million muslim feel real citiz...,0.9875
1,1,eib9ut,worldnews,skippy_smooth,2020-01-01 00:21:23,bgr.com,https://bgr.com/2019/12/31/china-pneumonia-sic...,A scary unidentified virus is spreading in China,70,,1,"[removed]...Ah, a fellow time traveler....Well...","Dozens of residents of Wuhan, the capital city...",scary unidentified virus spread china,01-2020,0,fellow time traveler good turn interest gamepl...,0.9705
2,2,eibebq,worldnews,vannybros,2020-01-01 00:31:16,cnn.com,https://www.cnn.com/2019/12/31/politics/north-...,Kim Jong Un warns hostile US policy means ther...,10,,1,This guy...Trump will comfort Kim....Being tha...,Washington CNN —\n\nNorth Korean Leader Kim Jo...,kim jong warn hostile policy mean korean penin...,01-2020,14,guy trump comfort kim intentions nuke meaningl...,-0.7849
3,3,eibo17,worldnews,OldFashionedJizz,2020-01-01 00:53:25,theguardian.com,https://www.theguardian.com/world/2019/dec/31/...,"North Korean leader to end missile test ban, c...",7,,1,[removed]...That's a choice of course. We can ...,Kim Jong-un has signalled that North Korea wil...,north korean leader end missile test ban claim...,01-2020,14,choice course agree big problem north korea pu...,-0.9791
4,4,eibo8f,worldnews,didnt_see_that_tree,2020-01-01 00:53:55,independent.co.uk,https://www.independent.co.uk/life-style/gadge...,Snapchat has stopped working for users around ...,8,,1,Oh The Humanity ! ! !...OMG....what will teena...,For free real time breaking news alerts sent s...,snapchat stop work users world tens thousands ...,01-2020,26,humanity omg teenagers world snap corner carpe...,-0.8176


In [89]:
sentiment_analyzer = SentimentIntensityAnalyzer()
df["comment_sentiment"] = df['cleaned_comments'].apply(lambda x: sentiment_analyzer.polarity_scores(x)["compound"])

In [90]:
df['created_utc'] = pd.to_datetime(df['created_utc'])
df["created_dt_month"] = df['created_utc'].dt.strftime('%m-%Y')

In [91]:
sentiment_df = df[["created_dt_month", "topic", "comment_sentiment"]].groupby(["created_dt_month", "topic"]).mean()
sentiment_df.head()

comment_sentiment
created_dt_month topic                   
01-2020          -1             -0.252900
                  0             -0.162995
                  1             -0.032364
                  2             -0.519639
                  3             -0.313533

In [92]:
sentiment_df = sentiment_df.reset_index()

In [93]:
map_topic = freq[["Topic", "Name"]]
map_topic.columns = ['topic', 'name']
merged_df = sentiment_df.merge(map_topic, on=['topic'])

In [94]:
merged_df["created_dt_month"] = pd.to_datetime(merged_df["created_dt_month"])
merged_df = merged_df.sort_values("created_dt_month")
merged_df = merged_df[merged_df['topic']!=-1]
merged_df.head()

,created_dt_month,topic,comment_sentiment,name
30,2020-01-01,5,0.051789,5_mask_face mask_face_ventilators
130,2020-01-01,25,0.162744,25_climate_carbon_emissions_climate change
35,2020-01-01,6,-0.114862,6_coronavirus_case_coronavirus case_pandemic
120,2020-01-01,23,-0.433875,23_iran_coronavirus_sanction_iran coronavirus
195,2020-01-01,38,-0.234440,38_earthquake_magnitude_magnitude earthquake_p...


In [95]:
plot_df = merged_df[merged_df['topic'].isin([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])]
# plot_df["created_dt_month"] = pd.to_datetime(plot_df["created_dt_month"])
fig = px.line(plot_df, x="created_dt_month", y="comment_sentiment", color="name")
fig.show()
# sns.lineplot(data=plot_df, x="created_dt_month", y="comment_sentiment", hue="name")

In [96]:
fig.write_html("/content/drive/MyDrive/news_comments_sentiments.html")

In [97]:
merged_df.to_csv("/content/drive/MyDrive/news_comment_sentiment.csv", index=False)

In [98]:
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
docs = titles
# topic_model = BERTopic(verbose=True, n_gram_range=(1, 3),nr_topics='auto',min_topic_size=100)
# topics, _ = topic_model.fit_transform(docs)

# Preprocess Documents
documents = pd.DataFrame({"Document": docs,
                          "ID": range(len(docs)),
                          "Topic": topics})
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
cleaned_docs = topic_model._preprocess_text(documents_per_topic.Document.values)

# Extract vectorizer and analyzer from BERTopic
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
               for topic in range(len(set(topics))-1)]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_v')
coherence = coherence_model.get_coherence()
print(coherence)

0.5704927234088514
